In [1]:
from path import path
import os
rootdir = path(u'/home/pfernique/Desktop/PyClangLite')
for filepath in (rootdir/'src'/'wrapper').walkfiles('*.[hc]*'):
    os.remove(filepath.abspath())

In [2]:
import subprocess
popen = subprocess.Popen(['clang++', '-x', 'c++', '-v', '-E', '/dev/null'], stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
out, err = popen.communicate()
sysincludes = err.splitlines()
headers = []
flags = ['-x', 'c++', '-std=c++11', '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS']
if '#include <...> search starts here:' not in sysincludes or 'End of search list.' not in sysincludes:
    warnings.warn('System includes not computed: parsing clang command output failed', Warning)
else:
    lindex = sysincludes.index('#include <...> search starts here:')
    rindex = sysincludes.index('End of search list.')
    sysincludes = sysincludes[lindex+1:rindex]
flags.extend(['-I'+str(path(sysinclude.strip()).abspath()) for sysinclude in sysincludes])
for sysinclude in sysincludes:
    sysinclude = path(sysinclude.strip())
    clanginclude = sysinclude/'clang'
    if clanginclude.exists():
        headers.extend([sysinclude.relpathto(header) for header in clanginclude.walkfiles('*.h')])
selfcontained = rootdir/'src'/'wrapper'/'clang.h'
selfcontained.write_lines(['#include <' + str(header) + '>' for header in headers])
headers  = [selfcontained]

In [3]:
from autowig import autowig
import pickle
try:
    with open('pyclanglite.pkl', 'r') as f:
        asg = pickle.load(f)
except:
    asg = autowig.AbstractSemanticGraph()
    autowig.parser.plugin = 'pyclanglite'
    autowig.parser(asg, headers, flags=flags, bootstrap=False)
    with open('pyclanglite.pkl', 'w') as f:
        pickle.dump(asg, f)

In [4]:
controller = """
def bootstrap_controller(asg):
    for node in asg.nodes('::clang::ASTUnit::top_level_(begin|end)'):
        if not node.return_type.boost_python_export:
            node.return_type.boost_python_export = True
    for node in asg.nodes('^class ::std::(shared|weak|unique)_ptr$'):
        node.is_copyable = False
        node.is_smart_pointer = True
    node = asg.nodes('::clang::HeaderSearch::NormalizeDashIncludePath').pop()
    node.boost_python_export = False
    for node in asg.nodes('::clang::DeclSpec::(TSCS|TSW|TSS|TST)_'):
        node.boost_python_export = False
"""
exec controller + "    return asg"
autowig.controller['bootstrap'] = bootstrap_controller
autowig.controller.plugin = 'bootstrap'
asg = autowig.controller(asg)

In [5]:
subset = asg.nodes('::clang::tooling::buildASTFromCodeWithArgs')
classes = [asg['class ::clang::Type'], asg['class ::clang::Decl']]
subset += classes
subset += classes[0].subclasses(recursive=True)
subset += classes[1].subclasses(recursive=True)

In [6]:
from autowig.boost_python_generator import caching
caching(asg)

In [ ]:
autowig.generator.plugin = 'boost_python'
wrappers = autowig.generator(asg, subset,
                             module=rootdir/'src'/'wrapper'/'_pyclanglite.cpp',
                             decorator=None, closure=True)
if autowig.parser == 'libclang':
    pass

In [ ]:
for wrapper in wrappers:
    wrapper.write()
session = autowig.scons(rootdir, '-k', '-j6')
session

In [ ]:
while session.err:
    controller += '\n' + autowig.feedback(session, rootdir, asg,
                                          variantdir='build-scons',
                                          tabsize=4, indent=1,
                                          force=True)
    exec controller + "\n    return asg"
    autowig.controller['bootstrap'] = bootstrap_controller
    autowig.controller.plugin = 'bootstrap'
    asg = autowig.controller(asg)
    for wrapper in wrappers:
        if wrapper.on_disk:
            wrapper.write()
    session = autowig.scons(rootdir, '-k', '-j6')
session

In [ ]:
print controller

In [ ]:
asg['class ::clang::DeclContext::decl_iterator'].methods()